In [1]:
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1759026332,2025-09-27 23:25,1759025700,2025-09-27 23:15,13.2,55.8,0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/night/116.png,1003,6.3,10.1,85,E,1024.0,30.24,0.0,0.0,94,75,12.4,54.4,9.5,49.1,10.7,51.3,9.4,48.9,10.0,6.0,0.0,8.8,14.1,"[{'date': '2025-09-27', 'date_epoch': 17589312..."


In [4]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-09-27,1758931200,"[{'time_epoch': 1758942000, 'time': '2025-09-2...",20.5,69.0,7.7,45.8,12.7,54.9,8.7,14.0,0.02,0.0,0.0,8.0,4.0,80,0,0,0,0,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/day/116.png,1003,2.6,06:01 AM,06:15 PM,09:14 AM,11:35 PM,Waxing Crescent,23,0,0


In [5]:
df_hour = pd.json_normalize(hour_list)

# Converter para datetime (se ainda não estiver)
df_hour["time"] = pd.to_datetime(df_hour["time"])

# Definir como índice
df_hour = df_hour.set_index("time")

# Selecionar apenas colunas numéricas
df_numeric = df_hour.select_dtypes(include="number")

# Reamostrar de 5h em 5h e calcular média
df_5h = df_numeric.resample("5h").mean().reset_index()

print(df_5h.head())

df_5h

NameError: name 'hour_list' is not defined

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)


Error: Not connected to OpenGRID Jupyter server

In [ ]:
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
]

df_limpo = df_5h[[c for c in colunas_importantes if c in df_5h.columns]]
df_limpo.sample(5)

Error: Not connected to OpenGRID Jupyter server

In [ ]:
df_tabela1 = pd.DataFrame({
    "estilos": ["casual", "esportivo", "social", "streetwear"],
    "data e hora": df_limpo["time"].values[:4],
    "temperatura": df_limpo["feelslike_c"].values[:4],
    "chance de chuva": df_limpo["chance_of_rain"].values[:4],
    "umidade": df_limpo["humidity"].values[:4],
    "uv": df_limpo["uv"].values[:4],
})

df_tabela1

Error: Not connected to OpenGRID Jupyter server

In [ ]:
# Garante que 'data e hora' seja do tipo datetime
df_tabela1['data e hora'] = pd.to_datetime(df_tabela1['data e hora'])

# Extrai hora, minuto, dia, etc.
df_tabela1['hora'] = df_tabela1['data e hora'].dt.hour
df_tabela1['dia'] = df_tabela1['data e hora'].dt.day
df_tabela1['mes'] = df_tabela1['data e hora'].dt.month

Error: Not connected to OpenGRID Jupyter server

In [ ]:
print(df_tabela1.columns.tolist())

Error: Not connected to OpenGRID Jupyter server

In [ ]:
#transformando variáveis e em um número categorico 

X = df_tabela1[['temperatura', 'chance de chuva', 'umidade', 'uv', 'hora']]  # todas numéricas
y = df_tabela1['estilos']  # variável alvo

Error: Not connected to OpenGRID Jupyter server

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)


Error: Not connected to OpenGRID Jupyter server

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)


Error: Not connected to OpenGRID Jupyter server

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier(max_depth=4, random_state=42)
clf.fit(X_train, y_train)


Error: Not connected to OpenGRID Jupyter server

In [ ]:
# Supondo que df_tabela1 já tenha a coluna 'hora' extraída
X_dummies = df_tabela1[['temperatura', 'chance de chuva', 'umidade', 'uv', 'hora']]

Error: Not connected to OpenGRID Jupyter server

In [ ]:
df_tabela1['predicao_estilo'] = clf.predict(X_dummies)
print(df_tabela1)


Error: Not connected to OpenGRID Jupyter server